In [1]:
import overpy

# Example
Find it [here](https://github.com/DinoTools/python-overpy).

In [48]:
api = overpy.Overpass()

# fetch all ways and nodes
result = api.query("""
    way(50.746,7.154,50.748,7.157) ["highway"];
    (._;>;);
    out body;
    """)

# Road of interest
This is to get information about the exact road that I'm looking at. It will most likely return a segment that is longer than I want, which is fine.

I can pull things like type of highway, surface, and tiger parameters (are those zip codes in there?).

In [3]:
#road_query = 'way[highway](around:5,44.15349716748267, -72.49253549908933);(._;>;);out body;'
road_query = 'way(around:5, 44.230271753139036, -72.56361378170132) ["highway"];(._;>;);out meta;'

In [4]:
road_result = api.query(road_query)

In [5]:
len(road_result.ways)

1

In [6]:
len(road_result.nodes)

83

In [7]:
road_result.ways[0].tags

{'highway': 'tertiary',
 'name': 'Paine Turnpike North',
 'surface': 'asphalt',
 'tiger:cfcc': 'A41',
 'tiger:county': 'Washington, VT',
 'tiger:name_base': 'Paine',
 'tiger:name_direction_suffix': 'N',
 'tiger:name_type': 'Tpke',
 'tiger:reviewed': 'no',
 'tiger:zip_left': '05602',
 'tiger:zip_left_1': '05602',
 'tiger:zip_left_2': '05602',
 'tiger:zip_right': '05602',
 'tiger:zip_right_1': '05602'}

In [8]:
len(road_result.nodes[0].tags.keys())

0

In [9]:
for way in road_result.ways:
    print("Name: %s" % way.tags.get("name", "n/a"))
    print("  Highway: %s" % way.tags.get("highway", "n/a"))
    print("  Nodes:")
    '''for node in way.nodes:
        print("    Lat: %f, Lon: %f" % (node.lat, node.lon))'''

Name: Paine Turnpike North
  Highway: tertiary
  Nodes:


# Road Area
This one is going to be gnarly. I have to decide how to deal with multiple things of varying types. The issue is that I have to know *a prior* what the tags of these features are going to be. 

So far I haven't found a good way for `overpy` to return an overall feature name. Maybe I could combine `overpass` and `overpy`. I could do the request with `overpy` see if I can pull out feature names there and then parse the results with `overpy`. I like the way `overpy` deals with results, it's like a pre-preprocessing step.

What I think is important:
- Roads
    - how many within the box?
    - does the type of these roads matter?
- Water
    - How many in the box?
    - Type matter?
- Land use
    - Farmland (include farmyard, orchard, vineyard)
    - forest (there are other things like meadow)
    - City (what are those features?)
- Barriers
    - Walls, fences, etc. (feature names needed)
- 

In [10]:
area_query = 'nwr(poly:"44.21942752883122 -72.56043380096837 44.229099324639975 -72.5561157214792 44.23530459506124 -72.55586883955655 \
                     44.237015175914344 -72.55395429014412 44.24820086817967 -72.56394830277897 44.240305872836885 -72.57175694223717 \
                     44.23556179577818 -72.57255065310554 44.23162010716889 -72.57106247361077 44.219529508340614 -72.57718734499564 \
                     44.21276642403039 -72.56379851791435 44.21942752883122 -72.56043380096837");out;'

In [11]:
area_result = api.query(area_query)

In [35]:
type(area_result)

overpy.Result

In [36]:
len(area_result.ways)

303

In [37]:
len(area_result.nodes)

2319

In [38]:
len(area_result.areas)

0

In [41]:
area_result.ways[0].tags

{'highway': 'service', 'surface': 'asphalt'}

In [21]:
relation = area_result.relations[0]

In [34]:
area_result.ways[0].__dict__

{'_result': <overpy.Result at 0x7f269c4d9a10>,
 'attributes': {},
 'id': 19715377,
 'tags': {'highway': 'service', 'surface': 'asphalt'},
 '_node_ids': [204888886, 204888887, 204888888],
 'center_lat': None,
 'center_lon': None}

In [27]:
relation.__dict__

{'_result': <overpy.Result at 0x7f269c4d9a10>,
 'attributes': {},
 'id': 8898132,
 'tags': {'admin_level': '8',
  'boundary': 'administrative',
  'name': 'Berlin',
  'type': 'boundary',
  'wikidata': 'Q821245'},
 'members': [<overpy.RelationWay ref=641779005 role=outer>,
  <overpy.RelationWay ref=641779008 role=outer>,
  <overpy.RelationWay ref=641779014 role=outer>,
  <overpy.RelationWay ref=39209882 role=outer>,
  <overpy.RelationWay ref=641778999 role=outer>,
  <overpy.RelationWay ref=641779000 role=outer>,
  <overpy.RelationWay ref=641707012 role=outer>,
  <overpy.RelationWay ref=641779006 role=outer>,
  <overpy.RelationWay ref=641707010 role=outer>,
  <overpy.RelationNode ref=158855906 role=admin_centre>],
 'center_lat': None,
 'center_lon': None}

In [26]:
for relation in area_result.relations:
    print(relation.tags)

{'admin_level': '8', 'border_type': 'city', 'boundary': 'administrative', 'is_in': 'USA, Vermont', 'is_in:country': 'USA', 'is_in:country_code': 'US', 'is_in:iso_3166_2': 'US:VT', 'is_in:state': 'Vermont', 'is_in:state_code': 'VT', 'name': 'Montpelier', 'place': 'city', 'source': 'TIGER/Line® 2008 Place Shapefiles (http://www.census.gov/geo/www/tiger/)', 'tiger:CLASSFP': 'C5', 'tiger:CPI': 'Y', 'tiger:FUNCSTAT': 'A', 'tiger:LSAD': '25', 'tiger:MTFCC': 'G4110', 'tiger:NAME': 'Montpelier', 'tiger:NAMELSAD': 'Montpelier city', 'tiger:PCICBSA': 'N', 'tiger:PCINECTA': 'N', 'tiger:PLACEFP': '46000', 'tiger:PLACENS': '01462152', 'tiger:PLCIDFP': '5046000', 'tiger:STATEFP': '50', 'type': 'boundary', 'wikidata': 'Q26426', 'wikipedia': 'en:Montpelier, Vermont'}
{'name': 'VT 62', 'network': 'US:VT', 'ref': '62', 'route': 'road', 'type': 'route'}
{'building': 'yes', 'type': 'multipolygon'}
{'natural': 'wood', 'type': 'multipolygon'}
{'admin_level': '8', 'boundary': 'administrative', 'name': 'Berli

In [ ]:
test_way = area_result.ways[2]

In [ ]:
for way in area_result.ways:
    print("Tags: %s" % way.tags)

In [ ]:
area_result.areas

# Combining `overpass` and `overpy`

In [203]:
import overpass
import numpy as np

In [204]:
api_pass = overpass.API()

In [205]:
area_query = 'nwr(poly:"44.21942752883122 -72.56043380096837 44.229099324639975 -72.5561157214792 44.23530459506124 -72.55586883955655 \
                     44.237015175914344 -72.55395429014412 44.24820086817967 -72.56394830277897 44.240305872836885 -72.57175694223717 \
                     44.23556179577818 -72.57255065310554 44.23162010716889 -72.57106247361077 44.219529508340614 -72.57718734499564 \
                     44.21276642403039 -72.56379851791435 44.21942752883122 -72.56043380096837");out;'

response = api_pass.get(area_query)

In [206]:
api_pass.get?

Signature: api_pass.get(query, responseformat='geojson', verbosity='body', build=True)
Docstring: Pass in an Overpass query in Overpass QL.
File:      ~/.anaconda3/envs/inaturalist/lib/python3.7/site-packages/overpass/api.py
Type:      method


In [42]:
area_result.test area_result.from_json

Signature: area_result.from_json(data, api=None)
Docstring:
Create a new instance and load data from json object.

:param data: JSON data returned by the Overpass API
:type data: Dict
:param api:
:type api: overpy.Overpass
:return: New instance of Result object
:rtype: overpy.Result
File:      ~/repos/python-overpy/overpy/__init__.py
Type:      method


In [149]:
from pandas.io.json import json_normalize
df = json_normalize(response["features"])

In [209]:
type(response)

geojson.feature.FeatureCollection

# Try reading with `geopandas`

In [98]:
import geopandas
import pandas as pd

In [71]:
geopandas.GeoDataFrame?

Init signature: geopandas.GeoDataFrame(*args, **kwargs)
Docstring:     
A GeoDataFrame object is a pandas.DataFrame that has a column
with geometry. In addition to the standard DataFrame constructor arguments,
GeoDataFrame also accepts the following keyword arguments:

Parameters
----------
crs : str (optional)
    Coordinate system
geometry : str or array (optional)
    If str, column to use as geometry. If array, will be set as 'geometry'
    column on GeoDataFrame.
File:           ~/.anaconda3/envs/inaturalist/lib/python3.7/site-packages/geopandas/geodataframe.py
Type:           type
Subclasses:     


In [210]:
geodf = geopandas.GeoDataFrame.from_features(response)

In [211]:
#culled_df = df.dropna(axis='index', how='all', subset=df.columns[4:])
culled_geodf = geodf.dropna(axis='index', how='all', subset=geodf.columns[1:])

In [189]:
columns = [
#'geometry', 
'power', 
'amenity', 
#'ele', 
#'gnis:county_id', 
#'gnis:created', 
#'gnis:feature_id', 
#'gnis:state_id', 
#'name', 
#'addr:state', 
#'gnis:county_name', 
#'gnis:import_uuid', 
#'gnis:reviewed', 
#'source', 
'building', 
#'addr:housenumber', 
#'addr:street', 
#'addr:city', 
#'addr:postcode', 
#'capacity', 
#'operator', 
#'phone', 
#'social_facility', 
#'social_facility:for', 
#'website', 
#'access', 
#'entrance', 
'highway', 
'surface', 
'service', 
#'ref', 
#'voltage', 
#'tiger:cfcc', 
#'tiger:county', 
#'tiger:reviewed', 
#'tiger:source', 
#'tiger:tlid', 
#'tiger:upload_uuid', 
#'oneway', 
#'tiger:name_base', 
#'tiger:name_type', 
#'tiger:zip_left', 
#'tiger:zip_right', 
#'expressway', 
#'lanes', 
#'maxspeed', 
#'tiger:name_direction_suffix', 
#'tiger:zip_left_1', 
#'tiger:zip_left_2', 
#'tiger:zip_right_1', 
'boundary', 
#'building:levels', 
#'shop', 
#'beds', 
#'emergency', 
'healthcare', 
#'old_name', 
#'opening_hours', 
#'wikidata', 
#'wikipedia', 
'parking', 
#'addr:country', 
#'denomination', 
#'religion', 
'landuse', 
#'military', 
#'healthcare:speciality', 
'aeroway', 
#'faa', 
'natural', 
#'water', 
'waterway', 
#'wetland', 
#'ref:walmart', 
]

In [196]:
cols = [
    'aeroway',
    'amenity',
    'barrier',
    'boundary',
    'building',
    'healthcare',
    'highway',
    'landuse',
    'leisure',
    'man_made',
    'natural',
    'parking',
    'power',
    'railway',
    'route',
    'service',
    'surface',
    'tourism',
    'waterway'
]

In [212]:
culled_geodf['highway'].value_counts()

service          76
residential      50
tertiary         14
trunk            10
unclassified      6
tertiary_link     2
track             2
Name: highway, dtype: int64

In [197]:
culled_geodf['highway'].value_counts()

service          76
residential      50
tertiary         14
trunk            10
unclassified      6
tertiary_link     2
track             2
Name: highway, dtype: int64

In [198]:
type(culled_geodf)

geopandas.geodataframe.GeoDataFrame

In [106]:
test_dict1 = {}
for i in culled_geodf.columns[1:]:
    #print(i)
    test_dict1[i] = culled_geodf[i].unique()[~pd.isnull(culled_geodf[i].unique())]

In [201]:
def make_feature_dict(df, features):
    """
    
    Parameters
    ----------
    df : geopandas.geodataframe.GeoDataFrame
        Dataframe containing features pulled from overpass API
        
    features : list
        List of column names associated with high-level feature names
        employed by overpass/OSM. Reference here:
        https://wiki.openstreetmap.org/wiki/Map_Features
        
    Returns
    -------
    dict
        Nested dictionary of features with value counts as eventual values.
        
    """
    
    feature_dict = {}
    for feature in features:
        if feature in df.columns:
            series = df[feature].value_counts()
            feature_dict[feature] = { k:v for (k,v) in zip(series.index, series.values)}
        else:
            feature_dict[feature] = None
    return feature_dict

In [202]:
make_feature_dict(culled_geodf, cols)

{'aeroway': {'helipad': 2},
 'amenity': {'loading_dock': 4,
  'hospital': 4,
  'parking': 4,
  'school': 4,
  'library': 2,
  'social_facility': 2,
  'place_of_worship': 2},
 'barrier': None,
 'boundary': {'administrative': 2},
 'building': {'yes': 216,
  'house': 166,
  'retail': 4,
  'garage': 2,
  'hospital': 2},
 'healthcare': {'hospital': 4},
 'highway': {'service': 76,
  'residential': 50,
  'tertiary': 14,
  'trunk': 10,
  'unclassified': 6,
  'tertiary_link': 2,
  'track': 2},
 'landuse': {'commercial': 4,
  'military': 4,
  'retail': 4,
  'quarry': 2,
  'residential': 2},
 'leisure': None,
 'man_made': None,
 'natural': {'wood': 8, 'wetland': 2, 'water': 2},
 'parking': {'surface': 2},
 'power': {'pole': 40, 'tower': 18, 'line': 6, 'minor_line': 4},
 'railway': None,
 'route': None,
 'service': {'parking_aisle': 34, 'driveway': 6},
 'surface': {'asphalt': 122, 'unpaved': 4, 'paved': 2},
 'tourism': None,
 'waterway': {'stream': 4}}

In [191]:
test_dict1

{'power': {'pole': 40, 'tower': 18, 'line': 6, 'minor_line': 4},
 'amenity': {'loading_dock': 4,
  'hospital': 4,
  'parking': 4,
  'school': 4,
  'library': 2,
  'social_facility': 2,
  'place_of_worship': 2},
 'building': {'yes': 216,
  'house': 166,
  'retail': 4,
  'garage': 2,
  'hospital': 2},
 'highway': {'service': 76,
  'residential': 50,
  'tertiary': 14,
  'trunk': 10,
  'unclassified': 6,
  'tertiary_link': 2,
  'track': 2},
 'surface': {'asphalt': 122, 'unpaved': 4, 'paved': 2},
 'service': {'parking_aisle': 34, 'driveway': 6},
 'boundary': {'administrative': 2},
 'healthcare': {'hospital': 4},
 'parking': {'surface': 2},
 'landuse': {'commercial': 4,
  'military': 4,
  'retail': 4,
  'quarry': 2,
  'residential': 2},
 'aeroway': {'helipad': 2},
 'natural': {'wood': 8, 'wetland': 2, 'water': 2},
 'waterway': {'stream': 4}}

In [110]:
print('[')
for i in culled_geodf.columns:
    print("'{}', ".format(i), end='\n')
print(']')

[
'geometry', 
'power', 
'amenity', 
'ele', 
'gnis:county_id', 
'gnis:created', 
'gnis:feature_id', 
'gnis:state_id', 
'name', 
'addr:state', 
'gnis:county_name', 
'gnis:import_uuid', 
'gnis:reviewed', 
'source', 
'building', 
'addr:housenumber', 
'addr:street', 
'addr:city', 
'addr:postcode', 
'capacity', 
'operator', 
'phone', 
'social_facility', 
'social_facility:for', 
'website', 
'access', 
'entrance', 
'highway', 
'surface', 
'service', 
'ref', 
'voltage', 
'tiger:cfcc', 
'tiger:county', 
'tiger:reviewed', 
'tiger:source', 
'tiger:tlid', 
'tiger:upload_uuid', 
'oneway', 
'tiger:name_base', 
'tiger:name_type', 
'tiger:zip_left', 
'tiger:zip_right', 
'expressway', 
'lanes', 
'maxspeed', 
'tiger:name_direction_suffix', 
'tiger:zip_left_1', 
'tiger:zip_left_2', 
'tiger:zip_right_1', 
'boundary', 
'building:levels', 
'shop', 
'beds', 
'emergency', 
'healthcare', 
'old_name', 
'opening_hours', 
'wikidata', 
'wikipedia', 
'parking', 
'addr:country', 
'denomination', 
'religion', 
'landu

In [105]:
test_dict1

{'power': array(['pole', 'tower', 'line', 'minor_line'], dtype=object),
 'amenity': array(['school', 'library', 'social_facility', 'loading_dock', 'hospital',
        'parking', 'place_of_worship'], dtype=object),
 'ele': array(['201', '303', '175'], dtype=object),
 'gnis:county_id': array(['023'], dtype=object),
 'gnis:created': array(['05/27/2008'], dtype=object),
 'gnis:feature_id': array(['2427482', '1919019', '1919017', '1919018'], dtype=object),
 'gnis:state_id': array(['50'], dtype=object),
 'name': array(['Family Center of Washington County',
        'Central Vermont Hospital Medical Library',
        'Central Vermont Chamber of Commerce',
        'Woodridge Rehabilitation and Nursing', 'Fisher Road',
        "Stretch's Way", 'Granger Road', 'Pleasantview Street',
        'Shady Lane', 'Woodridge Drive', 'Valerie Avenue', 'Judson Drive',
        'Addison Drive', 'George Street', 'Maple Street', 'Berlin Street',
        'Green Mountain Drive', 'Spruce Street', 'Wheelock Street',

# Single Road

In [220]:
road_query = 'way[highway](around:5,44.15349716748267, -72.49253549908933);(._;>;);out meta;'
road_query = 'nwr(around:5,44.15349716748267, -72.49253549908933);(._;>;);out;'

road_response = api_pass.get(road_query)

In [221]:
road_geodf = geopandas.GeoDataFrame.from_features(road_response)

In [222]:
#culled_df = df.dropna(axis='index', how='all', subset=df.columns[4:])
culled_road = road_geodf.dropna(axis='index', how='all', subset=road_geodf.columns[1:])

In [223]:
culled_road.shape

(2, 9)

In [224]:
road_geodf.shape

(50, 9)

In [225]:
culled_road

,geometry,highway,name,surface,tiger:cfcc,tiger:county,tiger:name_base,tiger:name_type,tiger:reviewed
24,POINT (-72.493351 44.155078),residential,Graniteville Road,asphalt,A41,"Washington, VT",Graniteville,Rd,no
49,POINT (-72.493351 44.155078),residential,Graniteville Road,asphalt,A41,"Washington, VT",Graniteville,Rd,no
